# Quantum Arithmetic Circuits

<a href="https://gitee.com/mindspore/docs/blob/master/docs/mindquantum/docs/source_en/quantum_arithmetic_circuit.ipynb" target="_blank"><img src="https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source_en.png"></a>

## Summary

This tutorial includes the implementation of 4-bit VBE quantum adder, subtractor, modular adder, modular multiplier and modular exponentiation circuit with Mindquantum. There is a progressive relationship between them, i.e., modular adder is based on adder, modular multiplier is based on modular adder and modular exponentiation circuit is based on multiplier multipliers.

The function of arithmetic circuits can be summarised as follow:

* adder, giving output: $a+b$;
* subtractor, output: $b-a$;
* modular adder, output: $(a + b) mod N$;
* modular mulplier, output: $a x mod N$;
* modular exponentiation circuit, output: $a^x mod N$.

while $a, b, x, N$ are some non-negtive integrals.

## Introduction

The development of quantum computing field has is now on the rise due to the technology advances and the computational advantages of some quantum algorithms over their classical counterpart. One of the most influential quantum algorithms is Shor's algorithm for integer factorization problem which offers super-polynomial speedup, threating the security of widely-used RSA cryptosystems when large number of qubits exist in the future. The critical circuit of Shor's algorithm is modular exponentiation circuit. Vedral, Barenco and Ekert (VBE) proposed earliest how to build quantum arithmetic circuits, including the modular exponentiation circuit, here we give the implementations of them.

This tutorial is mainly the implementation of paper: [H. T. Larasati and H. Kim, "Simulation of Modular Exponentiation Circuit for Shor's Algorithm in Qiskit, 2020](https://ieeexplore.ieee.org/document/9310794).

## Arithmetic Circuits

### Adder & Subtrator

The figure numbers in this tutorial coincides with original paper to help reader refer to them. VBE adder which maps quantum state $|a, b\rangle \rightarrow |a, a+b\rangle$ is shown in `Fig.1` . Three registers are required: Register A and B, each to contain the input a and b, respectively, and a carry register to hold the carry value during computation. The sum is store at register S($s_3 s_2 s_1 s_0$). $3n+1$ qubits is required for the for a n-bit adder. The CARRY, SUM and Reverse CARRY blocks in figure convert the binary logic to controlled gates in quantum circuit. Their functions are calculating carry, sum and uncompute carry. e.g., for the first vertical line in the first CARRY block，the carry bit(forth line) is $1$ only when $a_0, b_0$ are both $1$.

![Fig1_3bit_adder](images/Fig1_3bit_adder.jpg)

The implementation of 4-bit adder circuit is shown in `Fig.6` . When applied backwards, this circuit performs the subtraction $|a, b\rangle \rightarrow |a, b-a\rangle$. When $b < a$ , meaning that
subtraction will yield negative number, the circuit will output the result in two’s complements form: the n+1 bit will be in state $|1\rangle$ in this case. This reversed circuit, which is also the inverse for this case, is often referred to as *subtractor*, *reversed adder*, or $adder^{-1}$. A implementation of subtractor is drawn in `Fig.7` .

![Fig6_adder](images/Fig6_adder.jpg)

![Fig7_subtractor](images/Fig7_subtractor.jpg)

Here we show the code of adder( `Fig.6` ) and subtractor( `Fig.7` ) with Mindquantum.


In [ ]:
from mindquantum.core.gates import X, SWAP
from mindquantum.core.circuit import Circuit, shift, UN

In [3]:
def Adder() -> Circuit:
    """4-bit adder, from Fig.6
    """
    return Circuit([
        X(10, [4, 0]),
        X(11, [5, 1]),
        X(12, [6, 2]),
        X(8, [7, 3]),
        X(4, 0),
        X(5, 1),
        X(6, 2),
        X(7, 3),
        X(10, [9, 4]),
        X(11, [10, 5]),
        X(12, [11, 6]),
        X(8, [7, 12]),
        X(7, 12),
        X(12, [11, 6]),
        X(6, 2),
        X(12, [6, 2]),
        X(6, 2),
        X(6, 11),
        X(11, [10, 5]),
        X(5, 1),
        X(11, [5, 1]),
        X(5, 1),
        X(5, 10),
        X(10, [9, 4]),
        X(4, 0),
        X(10, [4, 0]),
        X(4, 0),
        X(4, 9)
    ])


def AdderRev() -> Circuit:
    """4-bit reversed adder (subtractor), from Fig.7
    """
    return Circuit([
        X(4, 9),
        X(4, 0),
        X(10, [4, 0]),
        X(4, 0),
        X(10, [9, 4]),
        X(5, 10),
        X(5, 1),
        X(11, [5, 1]),
        X(5, 1),
        X(11, [10, 5]),
        X(6, 11),
        X(6, 2),
        X(12, [6, 2]),
        X(6, 2),
        X(12, [11, 6]),
        X(7, 12),
        X(8, [7, 12]),
        X(7, 3),
        X(12, [6, 2]),
        X(8, [7, 3]),
        X(6, 2),
        X(12, [11, 6]),
        X(11, [5, 1]),
        X(5, 1),
        X(11, [10, 5]),
        X(10, [4, 0]),
        X(4, 0),
        X(10, [9, 4])
    ])

Invresed adder(subtractor) can be obtained easily from with method `reverse()` . e.g.:


In [4]:
def AdderRev2() -> Circuit:
    """Reversed-adder"""
    cir = Adder()
    cir.reverse()
    return cir

For convenience, here we give two functions: `c2q(...)` and `q2c(...)` , which convert the intergal to quantum circuit and measure quantum circuit to get a integral respectively.


In [5]:
def c2q(qlist: list, value: int) -> Circuit:
    """Convert integral `value` to specific bit of a quantum circuit.
    Args:
        qlist:  list[int]: Specific qubit to store the integral with smaller
                           digit at the end, such as [3,2,1,0].
        value:  int:       The non-negtive decimal integral.
    Return:
        Prepared circuit.
    """
    vbin = bin(value)[2:][::-1]
    qlist = qlist[::-1]
    cir = Circuit([X.on(q) for q, v in zip(qlist, vbin) if v == '1'])
    return cir


def q2c(cir: Circuit, qlist: list) -> int:
    """Measure the circuit to get the circuit state.
    Args:
        cir:   Quantum circuit.
        qlist: The bit list need to be measured, e.g. the [q2, q1, q0] is
               means int(q2q1q0).
    Return:
        The measure value in decimal way.
    """
    ket_str = cir.get_qs(ket=True)
    print(f'ket string: {ket_str}')
    res_str = ""
    for q in qlist:
        res_str += ket_str[-q-2]
    return int(res_str, base=2)

Now we verify the *adder* and *subtractor* with $a=3, b=8, a+b=11, b-a=5$. Here is the code.


In [6]:
def demo1():
    """Verify Fig.6 (adder), Fig.7 (subtractor)
    """
    print("Verify adder & subtractor:")
    # verify b + a, b - a
    a = 3
    b = 8
    # Assign `a` to the 3,2,1,0 circuit.
    cir_a = c2q([3, 2, 1, 0], a)
    cir_b = c2q([7, 6, 5, 4], b)

    cir_add = cir_a + cir_b + Adder()
    # Measure the quantum circuit to get the result.
    q_add = q2c(cir_add, [7, 6, 5, 4])

    cir_sub = cir_a + cir_b + AdderRev()
    q_sub = q2c(cir_sub, [7, 6, 5, 4])

    print(
        f"q_add = {q_add}, c_add = {b + a}\nq_sub = {q_sub}, c_sub = {b - a}\n")


demo1()

Verify adder & subtractor:
ket string: 1¦0000010110011⟩
ket string: 1¦0010001010011⟩
q_add = 11, c_add = 11
q_sub = 5, c_sub = 5



Where the `q_` at the beginning means `quantum` result, `c_` means classical (correct) value, `add` and `sub` mean adder and subtractor respectively, `ket string` is the final state of circuit. The circuit gains the correct answer for giving value.


### Modular Adder

Modular adder maps $|a, b\rangle \rightarrow |a, (a+b) mod N\rangle$. VBE modular adder makes use of the concept of two’s complements to detect the comparison. As shown in Fig. 2, it uses 5 adders (3 adders and 2 reversed adder to be specific, but we refer them generally as adders for this case), in which the first two are for performing $a+b-N$, the third is for adding back if the former operation yield negative values (indicated by the values stored in the Temp Qubit, which is controlled by the MSB of Register B, $b_{n+1}$), and another two are for restoring the temp qubit bit back to $|0\rangle$. As with the adder, the computation result is stored in Register B.

![Fig2_modular_adder](images/Fig2_modular_adder.jpg)

`Fig.8` presents the implementation of a 4-bit modular adder, in which gray lines at the first and last part of the circuit is used to separate the inputs with the “internal” part of the circuit.

![Fig8_modular_adder](images/Fig8_modular_adder.jpg)

The code of Modular adder and verification are as following.


In [7]:
def ModAdder() -> Circuit:
    """Modular adder, from Fig.8
    """
    cir = Circuit()
    cir += Adder()
    cir += UN(SWAP, maps_obj=[(0, 13), (1, 14), (2, 15), (3, 16)])
    cir += AdderRev2()
    cir += Circuit([X(8), X(17, 8), X(8)])
    cir += UN(X, maps_obj=[0, 1, 2, 3], maps_ctrl=[17, 17, 17, 17])
    cir += Adder()
    cir += UN(X, maps_obj=[0, 1, 2, 3], maps_ctrl=[17, 17, 17, 17])
    cir += UN(SWAP, maps_obj=[(0, 13), (1, 14), (2, 15), (3, 16)])
    cir += AdderRev2()
    cir += X(17, 8)
    cir += Adder()
    return cir


def demo2():
    """Verify modular adder
    """
    print("Verify modular adder:")
    # 1. Get (a + b) mod n
    # 2. Get (b - a) mod n
    a = 11
    b = 9
    n = 15
    # encoding the decimals
    cir_a = c2q([3, 2, 1, 0], a)
    cir_b = c2q([7, 6, 5, 4], b)
    cir_n = c2q([16, 15, 14, 13], n)
    # full adder circuit
    cir_madd = cir_a + cir_b + cir_n + ModAdder() + cir_n
    # measure to get the result
    q_add = q2c(cir_madd, [7, 6, 5, 4])
    # correct value
    c_add = (a + b) % n
    print(f"q_add = {q_add}, c_add = {c_add}")


demo2()

Verify modular adder:
ket string: 1¦000000000001011011⟩
q_add = 5, c_add = 5


### Modular Multiplier

Modular multiplier maps $|x, 0\rangle \rightarrow |x, ax mod N\rangle$. There is a control bit ($control_0$ or $|c\rangle$), if $control_0 = 0$，then the maps is $|x, 0\rangle \rightarrow |x, x\rangle$. Since

$$
ax = ax_{n-1} 2^{n-1} + \cdots + a x_1 2^1 + a x_0 2^0 = \sum_{x_k} a 2^k
$$

we can combine modular adder to make a multiplier. The structure of a multiplier is shown in `Fig.3` . Each `Modadd` assigns $a2^i mod N$ as input with Toffoli gate. e.g. $7x mod 15$, where $a=7$，$7\times 2^0 mod 15 = 7, 7\times 2^1 mod 15 = 14$, then the input of first `Modadd` is 7 (0111). $n$ bits multiplier needs $6n+3$ qubits in total. The input is assigned to $regx_0\sim regx_3$, modulo is assigned to $modulusN_0 \sim modulusN_3$ and output located at $b_0\sim b_4$. In `Fig.10` , the input is $x=3$, modulo $N=15$.

`Fig.10` shows the circuit to compute $7 x\mod 15$ where $x=3$. It may cost 20 seconds with a hardware of 8U+32G.

![Fig3_moduler_multiplier](images/Fig3_moduler_multiplier.jpg)

![Fig10_7xmod15](images/Fig10_7xmod15.jpg)

The code of modular multiplier and verification is as following.

In [8]:
def ModMulti7xmod15():
    """4-bit modular multiplier to compute 7x mod 15, from Fig.10
    """
    ns = 5  # number of shift bit
    cir = Circuit()
    cir += UN(X, maps_obj=[5, 6, 7], maps_ctrl=[[0, 1], [0, 1], [0, 1]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[5, 6, 7], maps_ctrl=[[0, 1], [0, 1], [0, 1]])

    cir += UN(X, maps_obj=[6, 7, 8], maps_ctrl=[[0, 2], [0, 2], [0, 2]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[6, 7, 8], maps_ctrl=[[0, 2], [0, 2], [0, 2]])

    cir += UN(X, maps_obj=[5, 7, 8], maps_ctrl=[[0, 3], [0, 3], [0, 3]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[5, 7, 8], maps_ctrl=[[0, 3], [0, 3], [0, 3]])

    cir += UN(X, maps_obj=[5, 6, 8], maps_ctrl=[[0, 4], [0, 4], [0, 4]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[5, 6, 8], maps_ctrl=[[0, 4], [0, 4], [0, 4]])

    cir += X(0)
    cir += UN(X, maps_obj=[9, 10, 11, 12],
              maps_ctrl=[[0, 1], [0, 2], [0, 3], [0, 4]])
    cir += X(0)
    return cir


def demo3():
    """Verify the modular multiplier
    """
    print("Verify the modular multiplier (about 20 seconds for 8U+32G computer):")
    # calculate 7x mod n
    ctrl = 1  # control bit
    x = 9
    n = 15
    cir_ctrl = c2q([0], ctrl)
    cir_x = c2q([4, 3, 2, 1], x)
    cir_n = c2q([21, 20, 19, 18], n)
    cir_7x = cir_ctrl + cir_x + cir_n + ModMulti7xmod15() + cir_n
    # measure to get result
    q_mul = q2c(cir_7x, [12, 11, 10, 9])
    c_mul = (7 * x) % 15

    print(f"q_mul = {q_mul}, c_mul = {c_mul}\n")


demo3()

Verify the modular multiplier (about 20 seconds for 8U+32G computer):
ket string: 1¦00000000000011000010011⟩
q_mul = 3, c_mul = 3



### Modular Exponentiation Circuit

Modular Exponentiation Circuit maps $|x, 1, 0\rangle = |x, a^x mod N, 0\rangle$, some $|0\rangle$ are auxiliary bits. Next we will see the theoretical part. Express $a$ and $x$ in binary mode:

$$
a = a_{n-1} 2^{n-1} +\cdots + a_1 2 + a_0
$$

$$
x = x_{n-1} 2^{n-1} +\cdots + x_1 2 + x_0
$$

then

$$
a^x = a^{x_{n-1}2^{n-1}} \times \cdots\times a^{x_1 2}\times a^{x_0}
$$

To realize the circuit, VBE proposes a quantum circuit as illustrated in `Fig.4` . The circuit consists of a series of modular multiplier pair (in a pair, one is the corresponding inverse circuit) with swap gates in the middle of each pair to preserve each multiplication result. The inverse circuit is for uncomputation of multiplication operation, preparing the circuit for the multiplication of the next bit. When two integrals $a, b$ satisfy $ab\equiv 1 mod N$, we can say $b$ is the inverse digit of $a$ for $N$, such as 13 is reverse digit of 7 for 15, since $7\times 13 mod 15 = 1$. In $CMODMULT^{-1}$ block, we first get the circuit of reverse digit, then reverse the circuit.

![Fig4_modular_exp](images/Fig4_modular_exp.jpg)

`Fig.12` is a 4-bit modular exponentiation circuit to compute $7^x mod 15$. The input lies in $regx_e xp_0 \sim regx_e xp_3$ and the output in $regx_0 \sim regx_3$. We can see $7^{2^0} mod 15 = 7^1 mod 15 = 7$，$7^{2^1} mod 15 = 4$，$7^{2^2} mod 15 = 1, 7^{2^3} mod 15 = 1$, and their inverses are $13, 4, 1, 1$ respectively, they are the values of multiplier in `Fig.12` .

![Fig12_modular_exp](images/Fig12_modular_exp.jpg)


Here we show the code of multipliers and their inverse multipliers used in `Fig.12` .


In [13]:
def ModMulti4xmod15():
    """4-bit mudular multiplier for 4x mod 15.
    """
    ns = 5  # number of shift
    cir = Circuit()
    cir += UN(X, maps_obj=[7], maps_ctrl=[[0, 1]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[7, 8], maps_ctrl=[[0, 1], [0, 2]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[8, 5], maps_ctrl=[[0, 2], [0, 3]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[5, 6], maps_ctrl=[[0, 3], [0, 4]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[6], maps_ctrl=[[0, 4]])
    cir += X(0)
    cir += UN(X, maps_obj=[9, 10, 11, 12],
              maps_ctrl=[[0, 1], [0, 2], [0, 3], [0, 4]])
    cir += X(0)
    return cir


def ModMulti1xmod15(ns=5):
    """4-bit mudular multiplier for 1x mod 15.
    """
    ns = 5  # number of shift
    cir = Circuit()
    cir += UN(X, maps_obj=[5], maps_ctrl=[[0, 1]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[5, 6], maps_ctrl=[[0, 1], [0, 2]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[6, 7], maps_ctrl=[[0, 2], [0, 3]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[7, 8], maps_ctrl=[[0, 3], [0, 4]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[8], maps_ctrl=[[0, 4]])
    cir += X(0)
    cir += UN(X, maps_obj=[9, 10, 11, 12],
              maps_ctrl=[[0, 1], [0, 2], [0, 3], [0, 4]])
    cir += X(0)
    return cir


def ModMulti13xmod15():
    """4-bit mudular multiplier for 13x mod 15.
    """
    ns = 5  # number of shift
    cir = Circuit()
    cir += UN(X, maps_obj=[5, 7, 8], maps_ctrl=[[0, 1], [0, 1], [0, 1]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[5, 7, 8], maps_ctrl=[[0, 1], [0, 1], [0, 1]])

    cir += UN(X, maps_obj=[5, 6, 8], maps_ctrl=[[0, 2], [0, 2], [0, 2]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[5, 6, 8], maps_ctrl=[[0, 2], [0, 2], [0, 2]])

    cir += UN(X, maps_obj=[5, 6, 7], maps_ctrl=[[0, 3], [0, 3], [0, 3]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[5, 6, 7], maps_ctrl=[[0, 3], [0, 3], [0, 3]])

    cir += UN(X, maps_obj=[6, 7, 8], maps_ctrl=[[0, 4], [0, 4], [0, 4]])
    cir += shift(ModAdder(), ns)
    cir += UN(X, maps_obj=[6, 7, 8], maps_ctrl=[[0, 4], [0, 4], [0, 4]])

    cir += UN(X, maps_obj=[8], maps_ctrl=[[0, 4]])
    cir += X(0)
    cir += UN(X, maps_obj=[9, 10, 11, 12],
              maps_ctrl=[[0, 1], [0, 2], [0, 3], [0, 4]])
    cir += X(0)
    return cir


def ModMulti4xmod15Rev():
    """Reversed 4-bit mudular multiplier for 4x mod 15.
    """
    cir = ModMulti4xmod15()
    cir.reverse()
    return cir


def ModMulti1xmod15Rev():
    """Reversed 4-bit mudular multiplier for 1x mod 15.
    """
    cir = ModMulti1xmod15()
    cir.reverse()
    return cir


def ModMulti13xmod15Rev():
    """Reversed 4-bit mudular multiplier for 13x mod 15.
    """
    cir = ModMulti13xmod15()
    cir.reverse()
    return cir

At last we give the code of modular exponentiation circuit in `Fig.12` which calculate $7^a mod 15$, then check the result with $a=3$.


In [ ]:
def ModExp():
    """4-bit modular exponentiation circuit, from Fig.12
    """
    ns = 4
    cir = Circuit()
    cir += X(4, 0)
    cir += X(5)
    cir += shift(ModMulti7xmod15(), ns)
    cir += UN(SWAP, maps_obj=[(5, 13), (6, 14), (7, 15), (8, 16)])
    cir += shift(ModMulti13xmod15Rev(), ns)

    cir += UN(X, maps_obj=[4, 4], maps_ctrl=[0, 1])
    cir += shift(ModMulti4xmod15(), ns)
    cir += UN(SWAP, maps_obj=[(5, 13), (6, 14), (7, 15), (8, 16)])
    cir += shift(ModMulti4xmod15Rev(), ns)

    cir += UN(X, maps_obj=[4, 4], maps_ctrl=[1, 2])
    cir += shift(ModMulti1xmod15(), ns)
    cir += UN(SWAP, maps_obj=[(5, 13), (6, 14), (7, 15), (8, 16)])
    cir += shift(ModMulti1xmod15Rev(), ns)

    cir += UN(X, maps_obj=[4, 4], maps_ctrl=[2, 3])
    cir += shift(ModMulti1xmod15(), ns)
    cir += UN(SWAP, maps_obj=[(5, 13), (6, 14), (7, 15), (8, 16)])
    cir += shift(ModMulti1xmod15Rev(), ns)

    cir += X(4, 3)
    return cir


def demo4():
    """Verify modular exponentiation circuit.
    """
    # calculate 7^a mod n
    print("Verify modular exponentiation circuit (about 20 minutes\
              for 8U+32G computer):")
    a = 3
    n = 15
    cir_a = c2q([3, 2, 1, 0], a)
    cir_n = c2q([25, 24, 23, 22], n)
    cir_expmod = cir_a + cir_n + ModExp() + cir_n

    q_res = q2c(cir_expmod, [8, 7, 6, 5])
    c_res = (7**a) % n

    print(f"q_res = {q_res}, c_res = {c_res}\n")


demo4()

Output：

```text
Verify modular exponentiation circuit (about 20 minutes for 8U+32G computer):
ket string: 1¦000000000000000000110100011⟩
q_res = 13, c_res = 13
```


## Future Work

The circuits provided in this tutorial is the baseline implementation without considering optimization. A more recent literatures have proposed various techniques to be more efficient, such as utilizing the semi-classical Fourier Transform (qubit recycling), various tinkering to the modular exponentiation circuit to reduce circuit depth and width, or utilizing a whole new approach using QFT-based circuit for performing exponentiation.

## Reference

[H. T. Larasati and H. Kim, "Simulation of Modular Exponentiation Circuit for Shor's Algorithm in Qiskit, 2020](https://ieeexplore.ieee.org/document/9310794)
